In [1]:
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
        .map(lambda line: line.split(","))
    #didn't work with take(1). believe returns 
    #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

def change_column_names(df, old_names, new_names):
    pass
    return df;

##input
##   string: 2013-02-22 21:38:45
##output: 
##   list: [u'2013', u'02', u'22', u'21', u'38', u'45']
def datetime_to_trace_time(timestamp):
    #strip double quotes
    timestamp = timestamp.replace('"', '')
    r_list = re.split('[- :]', timestamp)
    return [int(x) for x in r_list]

In [2]:
data_run='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_data_with_hr_spd.csv'
data_users='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_with_hr_users.csv'
df_runs = df_from_csv(data_run)
print type(df_runs)
df_runs.show(2)
df_runs.count()

<class 'pyspark.sql.dataframe.DataFrame'>
+--------------------+----------+------------+-------------+-------------+------------+-----------+--------+
|              "time"|"altitude"|"heart_rate"|   "latitude"|  "longitude"|     "speed"|"workoutid"|    "id"|
+--------------------+----------+------------+-------------+-------------+------------+-----------+--------+
|"2013-02-22 21:38...|          |   147.00000|51.4394768234|-0.8953504544|6.7140000000|  167479013|13856798|
|"2013-02-22 21:38...|          |   148.00000|51.4392022323|-0.8952703234|9.3492000000|  167479013|13856799|
+--------------------+----------+------------+-------------+-------------+------------+-----------+--------+
only showing top 2 rows



300000

In [3]:
sub_df = df_runs.limit(20)

In [4]:
#traces
import pandas as pd
import numpy as np
import re
import sys
import nbimporter
#for ordered dict for traces implementation
from collections import OrderedDict

import traces
from datetime import datetime
from datetime import timedelta
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.types import DoubleType
#for timedelta manipulation
from math import fabs

#kmeans
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import Vectors
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from  pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.linalg import Vectors, VectorUDT


from pyspark.sql.functions import *
#from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import DataFrame
from pyspark.sql.functions import count
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
from pyspark.mllib.linalg import Vectors
from pyspark.sql import HiveContext
from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'



from pyspark import SparkContext
#sc =SparkContext()


pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#%load_ext autotime
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#del min
#del max

data_run='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_data_with_hr_spd.csv'
data_users='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_with_hr_users.csv'

sqlContext = HiveContext(sc)



In [5]:
#get list of workoutids
#woutid_list = df_runs.select('"workoutid"').distinct()\
#    .rdd.map(lambda r:r[0]).collect()
#woutid_list

In [210]:
#class to create summary stats of workouts
class SummaryStat():
    data_points = ''
    new_col_names = []
    #list of tuples. 
    #   key: column name
    #   value: metric (sum/avg/etc)
    summary_conf = ''
    partition_name = ''
    def __init__(self, sql_df, summary_conf = [], \
                partition_name = ''):
        if partition_name:
            self.partition_name = partition_name
        else:
            self.partition_name = '"workoutid"'
        self.data_points = sql_df
        if not isinstance(sql_df, DataFrame):
            print 'please enter spark dataframe object'
            return
        #if summary_conf not defined, default to sum for all columns
        if not summary_conf:
            col_names = self.data_points.schema.names
            for i in col_names:
                summary_conf.append((i, 'sum'))
        else:
            if not isinstance(summary_conf, list):
                print 'check summary_conf type'
                return
            for element in summary_conf:
                if not isinstance(element, dict):
                    print 'check summary_conf type 2'
                    return
            self.summary_conf = summary_conf
        
    def describe(self):
        print self.data_points.limit(2)

    def get_new_col_names(self):
        return self.new_col_names
        
    def generate_summary(self):
        #rewnew new_col_names
        self.new_col_names = []
        #windows function test
        r_df = ''
        r_list_cols = []
        for dic in self.summary_conf:
            col_name = dic.keys()[0] #should only have 1 key per dictionary
            aggregation_type = dic[col_name]
            w = Window.partitionBy(self.partition_name)
            if aggregation_type == 'avg':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        avg(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())
                self.new_col_names.append(new_col_name)

            if aggregation_type == 'sum':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        sum(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())   
                self.new_col_names.append(new_col_name)

                
            if aggregation_type == 'max':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        max(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())    
                self.new_col_names.append(new_col_name)
                
            if aggregation_type == 'min':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        min(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct()) 
                self.new_col_names.append(new_col_name)
                
        for df in r_list_cols:
            if not r_df:
                r_df = df
            else:
                r_df = r_df.join(df, (r_df[self.partition_name]==df[self.partition_name]), 'inner')\
                    .drop(df[self.partition_name])
        self.new_dataframe = r_df
        return r_df
    


In [217]:
#minMaxScaler wrapper since originalMin/Max is only implemented in 2.0
class scaler_wrapper():
    mmModel = ''
    originalMin = ''
    originalMax = ''   
    
    def __init__(self, inputCol, outputCol):
        self.mmModel = MinMaxScaler(inputCol=inputCol, outputCol=outputCol)
        self.in_column = inputCol
        
    def get_input_col_name(self):
        return self.mmModel.getInputCol()

    def getMax(self):
        return self.mmModel.getMax()
        
    def getMin(self):
        return self.mmModel.getMin()
    
    def describe(self):
        print 'describe'
    
    def fit(self, df):
        col = self.mmModel.getInputCol()
        self.originalMin = df.select(col).rdd.flatMap(lambda x: x[0]).min()
        self.originalMax = df.select(col).rdd.flatMap(lambda x: x[0]).max()
        return self.mmModel.fit(df)
    
    #denormalize the value
    def denormalize(self, value):
        v = (value-self.getMin())*\
            (self.originalMax - self.originalMin)*\
            (self.getMax()-self.getMin()) + self.originalMin
        if v or v == 0:
            return v
        else:
            return -999
        
    def normalize(self, value):
        pass

In [216]:

#change column type
def change_type_cols_double(df, list_cols):
    for col_name in list_cols:
        df = df.withColumn(col_name, df[col_name].cast("double"))
    return df.na.fill(0)

#vectorize the column
#keeps the original name
def vectorize_columns(df, list_cols):
    tmp_col_name = 'temp'
    for col_name in list_cols:
        vectorize = udf(lambda vs: Vectors.dense(vs), VectorUDT())
        df = df.withColumn(tmp_col_name, vectorize(df[col_name])).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
    return df

##scaler wrapper usage
#generate normalized dataframe
#keep the original column names
#returns:
#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df(df, list_cols):
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    for col_name in list_cols:
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name)
        scalerModel = scaler.fit(df)
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #r_dict[col_name] = scaler
        r_dict[index] = scaler
        index+=1
    return df, r_dict

# MinMaxScaler doesn't have originalMin(only supported in 2.0). made one above cell with wrapper class
# #generate normalized dataframe
# #keep the original column names
# #returns:
# #   (df, {col_name:scalerModel, col_name:scalerModel,....})
# def normalize_df(df, list_cols):
#     tmp_col_name = 'temp'
#     r_dict = {}
#     for col_name in list_cols:
#         scaler = MinMaxScaler(inputCol=col_name, outputCol=tmp_col_name)
#         scalerModel = scaler.fit(df)
#         df = scalerModel.transform(df).drop(col_name)\
#             .withColumnRenamed(tmp_col_name, col_name)
#         r_dict[col_name] = [scaler, scalerModel]
#     return df, r_dict

#input: (MLLIB KMEANS)
#   cols_to_cluster: list of column names to cluster
# def cluster_summary_df(df, cols_to_cluster, num_clusters = 2):
#     vecAssembler = VectorAssembler(inputCols=cols_to_cluster,\
#         outputCol='features')
#     df_to_cluster = vecAssembler.transform(df.na.fill(0))
#     df_to_cluster = df_to_cluster.select('features')
#     df_to_cluster = df_to_cluster.rdd\
#         .map(lambda row : Vectors.dense([item for item in row]))
#     clusters = KMeans.train(df_to_cluster, num_clusters,\
#                            maxIterations = 10,\
#                         initializationMode="random")
    
#     return clusters


#ML KMEANS
#returns KMeanModel, df_to_cluster
def cluster_summary_df(df, cols_to_cluster, num_clusters = 2):
    vecAssembler = VectorAssembler(inputCols=cols_to_cluster,\
        outputCol='features')
    df_to_cluster = vecAssembler.transform(df.na.fill(0))
    df_to_cluster = df_to_cluster.select('features')
    #df_to_cluster = df_to_cluster.rdd\
     #   .map(lambda row : Vectors.dense([item for item in row]))
    kmeans = KMeans(k=num_clusters, seed=1)
    model = kmeans.fit(df_to_cluster)
    return model, df_to_cluster
    
#returns clusters. show cluster centerse by clusters.clusterCenters.
def cluster_df(df, cols_to_normalize, num_clusters = 2,  cols_to_cluster = []):
    if not cols_to_cluster:
        cols_to_cluster = cols_to_normalize
    df = change_type_cols_double(df, cols_to_normalize)
    df = vectorize_columns(df, cols_to_normalize)
    df, scalerModels = normalize_df(df, cols_to_cluster)
    r_obj = cluster_summary_df(df, cols_to_cluster, num_clusters)
    #r_obj = [KMeanModel, new_df]
    return r_obj[0], r_obj[1], scalerModels

def denormalize_centers(list_centers, scaler_models):
    new_centers = []
    for center in list_centers:
        n_center = []
        for c in range(len(center)):
            n_center.append(scaler_models[c].denormalize(center[c]))
        new_centers.append(n_center)
    return new_centers

# cluster section

In [213]:
#define columns to cluster on (they will be normalized)
#cols_to_cluster = ['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']
#create summary stats for workouts

s_conf = [{'"heart_rate"': 'avg'}, {'"speed"': 'avg'}, {'"altitude"': 'avg'}]

#obj = SummaryStat(df_runs, summary_conf=s_conf)
obj = SummaryStat(sub_df, summary_conf=s_conf)

#obj.describe()
sumstats = obj.generate_summary()
cols_to_cluster = obj.get_new_col_names()
print cols_to_cluster
sumstats.show()
sumstats.count()
#clusters = cluster_df(sumstats, cols_to_cluster, 5)
#clusters.clusterCenters

['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']
+------------------+-----------+-----------+--------------+
|  avg_"heart_rate"|avg_"speed"|"workoutid"|avg_"altitude"|
+------------------+-----------+-----------+--------------+
|             103.0|        0.0|  165504204|          null|
|150.06666666666666|   10.59552|  167479013|          null|
+------------------+-----------+-----------+--------------+



2

In [214]:
r_obj = cluster_df(sumstats, cols_to_cluster, num_clusters=2) 
kmodel = r_obj[0]
new_df = r_obj[1]
scaler_model_dict = r_obj[2]

In [215]:
print denormalize_centers(kmodel.clusterCenters(), scaler_model_dict)
print cols_to_cluster


[[103.0, 0.0, 0.0], [150.06666666666666, 10.59552, 0.0]]
['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']


# ---testing area

In [144]:
list_cols_to_change = ['"altitude"', '"heart_rate"', '"speed"']
clusters = cluster_df(df_runs, list_cols_to_change, 5)
clusters.clusterCenters

[array([ 0.02985281,  0.68377991,  0.05522858]),
 array([ 0.04080238,  0.60569136,  0.05948706]),
 array([ 0.03575399,  0.470584  ,  0.05394333]),
 array([ 0.03248862,  0.36756594,  0.04606859]),
 array([ 0.03308665,  0.53706301,  0.05880067])]

In [81]:
sub_df = df_runs.limit(20)
sub_df.head(2)

[Row("time"=u'"2013-02-22 21:38:45"', "altitude"=u'', "heart_rate"=u'147.00000', "latitude"=u'51.4394768234', "longitude"=u'-0.8953504544', "speed"=u'6.7140000000', "workoutid"=u'167479013', "id"=u'13856798'),
 Row("time"=u'"2013-02-22 21:38:57"', "altitude"=u'', "heart_rate"=u'148.00000', "latitude"=u'51.4392022323', "longitude"=u'-0.8952703234', "speed"=u'9.3492000000', "workoutid"=u'167479013', "id"=u'13856799')]

In [9]:
#probably smarter to go from top to down so creating summary will
#be O(n), where n is number of rows in df
s_conf = [{'"heart_rate"': 'avg'}, {'"speed"': 'avg'}, {'"altitude"': 'avg'}]

obj = SummaryStat(df_runs, summary_conf=s_conf)
#obj.describe()
sumstats = obj.generate_summary()
sumstats.show()
sumstats.count()

+------------------+------------------+-----------+------------------+
|  avg_"heart_rate"|       avg_"speed"|"workoutid"|    avg_"altitude"|
+------------------+------------------+-----------+------------------+
|            158.24| 9.455241600000008|  171117914|169.40080000000015|
|           148.138|11.577304799999983|  219630823| 41.31679999999993|
|           152.662|         7.3092888|  325622855|21.065600000000018|
|165.95172413793102| 7.994391724137932|  430188558| 289.2551724137944|
|           164.334|11.034640800000005|  325634772|245.03999999999948|
|           162.708| 9.122702400000005|  358388284| 88.63919999999996|
|           144.008|10.763495999999996|  513660945|-5.325999999999993|
|           151.172|12.929040000000002|  532228357| 82.64479999999995|
|           148.964|12.785579999999992|  628219304| 42.66640000000001|
|127.00862068965517| 8.785924137931035|  436651654|119.02758620689657|
| 145.0976430976431|10.043393939393928|  547204459| 1.528619528619532|
|150.8

714

In [ ]:
#get some column names that we want to cluster
print sumstats.describe()

In [ ]:
cols = []
# for s in s_conf:
#     for key in s:
#         cols.append(key)
# print cols

cols=['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']

clusters = cluster_summary_df(sumstats, cols, num_clusters = 8)
clusters.clusterCenters

In [124]:
print sub_df.schema
list_cols_to_change = ['"altitude"', '"heart_rate"', '"speed"']
newd = change_type_cols_double(sub_df, list_cols_to_change)

print '\n'


#vectorize = udf(lambda vs: Vectors.dense(vs), VectorUDT())
#df = df.withColumn('temp', vectorize(sub_df[col_name]))

dd = vectorize_columns(newd, list_cols_to_change)

#print dd.show()
#print '\n'
#normalize_df(dd, ['"speed"']).show()
#scaler = MinMaxScaler(inputCol='"speed"', outputCol="scaledspeed")
#scalerModel = scaler.fit(dd)
#scaledData = scalerModel.transform(dd)
#print scaledData.show()
normalize_df(dd, list_cols_to_change).show()
#normalize_df(dd, ['"speed"'])

StructType(List(StructField("time",StringType,true),StructField("altitude",StringType,true),StructField("heart_rate",StringType,true),StructField("latitude",StringType,true),StructField("longitude",StringType,true),StructField("speed",StringType,true),StructField("workoutid",StringType,true),StructField("id",StringType,true)))


+--------------------+-------------+-------------+-----------+--------+----------+--------------------+--------------------+
|              "time"|   "latitude"|  "longitude"|"workoutid"|    "id"|"altitude"|        "heart_rate"|             "speed"|
+--------------------+-------------+-------------+-----------+--------+----------+--------------------+--------------------+
|"2013-02-22 21:38...|51.4394768234|-0.8953504544|  167479013|13856798|     [0.5]|[0.9206349206349206]|[0.5020188425302826]|
|"2013-02-22 21:38...|51.4392022323|-0.8952703234|  167479013|13856799|     [0.5]|[0.9365079365079365]| [0.699057873485868]|
|"2013-02-22 21:39...|51.4390821196|-0.89535

In [31]:
vectorize = udf(lambda vs: Vectors.dense(vs), VectorUDT())
print Vectors.dense(sub_df['"speed"'])

ValueError: setting an array element with a sequence.

In [37]:
print sub_df.schema
sub_df = sub_df.withColumn('"speed"', sub_df['"speed"'].cast("double"))
print sub_df.schema

StructType(List(StructField("time",StringType,true),StructField("altitude",StringType,true),StructField("heart_rate",StringType,true),StructField("latitude",StringType,true),StructField("longitude",StringType,true),StructField("speed",StringType,true),StructField("workoutid",StringType,true),StructField("id",StringType,true)))
StructType(List(StructField("time",StringType,true),StructField("altitude",StringType,true),StructField("heart_rate",StringType,true),StructField("latitude",StringType,true),StructField("longitude",StringType,true),StructField("speed",DoubleType,true),StructField("workoutid",StringType,true),StructField("id",StringType,true)))


In [39]:
dd = sub_df.withColumn('speed', vectorize(sub_df['"speed"']))
#print dd.show()
print dd.schema

StructType(List(StructField("time",StringType,true),StructField("altitude",StringType,true),StructField("heart_rate",StringType,true),StructField("latitude",StringType,true),StructField("longitude",StringType,true),StructField("speed",DoubleType,true),StructField("workoutid",StringType,true),StructField("id",StringType,true),StructField(speed,VectorUDT,true)))


In [42]:
scaler = MinMaxScaler(inputCol='speed', outputCol="scaledspeed")
scalerModel = scaler.fit(dd)
scaledData = scalerModel.transform(dd)
print scaledData.show()

+--------------------+----------+------------+-------------+-------------+-------+-----------+--------+---------+--------------------+
|              "time"|"altitude"|"heart_rate"|   "latitude"|  "longitude"|"speed"|"workoutid"|    "id"|    speed|         scaledspeed|
+--------------------+----------+------------+-------------+-------------+-------+-----------+--------+---------+--------------------+
|"2013-02-22 21:38...|          |   147.00000|51.4394768234|-0.8953504544|  6.714|  167479013|13856798|  [6.714]|[0.5020188425302826]|
|"2013-02-22 21:38...|          |   148.00000|51.4392022323|-0.8952703234| 9.3492|  167479013|13856799| [9.3492]| [0.699057873485868]|
|"2013-02-22 21:39...|          |   148.00000|51.4390821196|-0.8953502029|10.6308|  167479013|13856800|[10.6308]| [0.794885598923284]|
|"2013-02-22 21:39...|          |   149.00000|51.4388703089|-0.8954045177| 13.374|  167479013|13856801| [13.374]|               [1.0]|
|"2013-02-22 21:39...|          |   151.00000|51.438684

# ------------testing area-------------

In [ ]:
#windows function test
w = Window.partitionBy('"workoutid"')
tmp = sub_df.select('"heart_rate"', '"workoutid"', \
              sum('"heart_rate"').over(w).alias("sum")\
                )

tmp[['"heart_rate"', '"workoutid"', 'sum']].show()

In [ ]:
#windows function test
w = Window.partitionBy('"workoutid"')
tmp = sub_df.select('"heart_rate"', '"speed"', '"workoutid"', \
              sum('"heart_rate"').over(w).alias("sum")\
              , avg('"speed"').over(w).alias("spd")\
                )

tmp[['"workoutid"', 'spd', 'sum']].distinct().show()

In [ ]:
sub_df.show()

In [ ]:
mmscaler = MinMaxScaler(inputCol = '"speed"', outputCol = 'scaled')
#scaled_df = mmscaler.fit(sub_df)
sub_df_wo_time = sub_df.drop('"time"').drop('"altitude"')
sub_df_wo_time = sub_df_wo_time.withColumn('"speed"', sub_df_wo_time['"speed"'].cast("double"))
#print sub_df_wo_time.na.fill(0).show()
#sub_df_wo_time = sub_df_wo_time.rdd.map(lambda row: Vectors.dense([item for item in row]))
print sub_df_wo_time.describe()
print sub_df_wo_time.show()
#row['"altitude"'] = 0
#for item in row:
#   print type(item), item

In [ ]:
# assembler = VectorAssembler(
#   inputCols=["DF_column"], outputCol="features"
# )

# assembled = assembler.transform(df)

# scaler = StandardScaler(
#   inputCol="features", outputCol="scaledFeatures",
#   withStd=True, withMean=False
# ).fit(assembled)

# scaler.transform(assembled)

assembler = VectorAssembler(\
            inputCols = ['"speed"'],
            outputCol = ['scaled'])
assembled = assembler.transform(sub_df_wo_time)

In [ ]:
scaler = MinMaxScaler(inputCol='"speed"', outputCol="scaledRevenue")
scalerModel = scaler.fit(sub_df_wo_time)
scaledData = scalerModel.transform(sub_df_wo_time)

In [20]:
sub_df.show()

+--------------------+----------+------------+-------------+-------------+-------------+-----------+--------+
|              "time"|"altitude"|"heart_rate"|   "latitude"|  "longitude"|      "speed"|"workoutid"|    "id"|
+--------------------+----------+------------+-------------+-------------+-------------+-----------+--------+
|"2013-02-22 21:38...|          |   147.00000|51.4394768234|-0.8953504544| 6.7140000000|  167479013|13856798|
|"2013-02-22 21:38...|          |   148.00000|51.4392022323|-0.8952703234| 9.3492000000|  167479013|13856799|
|"2013-02-22 21:39...|          |   148.00000|51.4390821196|-0.8953502029|10.6308000000|  167479013|13856800|
|"2013-02-22 21:39...|          |   149.00000|51.4388703089|-0.8954045177|13.3740000000|  167479013|13856801|
|"2013-02-22 21:39...|          |   151.00000|51.4386848174|-0.8952745982|12.6468000000|  167479013|13856802|
|"2013-02-22 21:39...|          |   152.00000|51.4385818038|-0.8951149229|11.6316000000|  167479013|13856803|
|"2013-02-

In [21]:
scaler = MinMaxScaler(inputCol='"heart_rate"', outputCol="scaledRevenue")

In [22]:
scalerModel = scaler.fit(sub_df)

IllegalArgumentException: u'requirement failed: Input column "heart_rate" must be a vector column'